In [1]:
import pandas as pd
import numpy as np

In [2]:
properties = pd.read_csv('DuvalSFR.csv')

C:\Users\robin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (21,63) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
print(properties.shape)
properties.head()

(252761, 105)


,RE,owner_1,owner_2,owner_3,mailing_address,mailing_city,mailing_state,property_use,subdivision,neighborhood,...,shed_yr,sun_rm,sunrm_ppu,sunrm_yr,additions,additions_effec_area,balcony_effec_area,garage_effec_area,fin_storage_effec_area,unfin_storage_effec_area
0,000006 0030,NADELEN ROBERT CARL JR,NADELEN ELIZABETH ANN,NaN,2503 SUMMERFIELD LN,JACKSONVILLE,FL,100,SECTION LAND,417300.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000007 0010,BRYCEVILLE LAND LLC,NaN,NaN,PO BOX 23637,JACKSONVILLE,FL,100,SECTION LAND,417300.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000009 0100,ENGEL ROBERT W,ENGEL GEORGEANN A,NaN,2495 U S 301 HWY N,BALDWIN,FL,100,SECTION LAND,417300.2,...,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000011 0000,BYRD FAMILY TRUST,BYRD JOEY B,BYRD LISA R,2425 US HWY 301 N,BALDWIN,FL,100,SECTION LAND,417300.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000012 0010,WHITTY MICHAEL S,WHITTY DEBORAH B,NaN,2204 US HIGHWAY 301 N,JACKSONVILLE,FL,100,SECTION LAND,417300.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#!pip install fuzzywuzzy
from fuzzywuzzy import fuzz

def remove_punctuation(word):
    word = word.replace('...','')
    word = word.replace('.','')
    word = word.replace(',','')
    word = word.replace('-','')
    word = word.replace(';','')
    word = word.replace("'",'')
    word = word.replace('"','')
    word = word.replace('/','')
    word = word.replace('(','')
    word = word.replace(')','')
    return word

def word_replacement(word1, word2):
    s = str()
    if fuzz.WRatio(word1,word2) > 84:
        s = word2
    else: 
        s = word1
    return s

for i in range(len(properties.mailing_city)):
    city = properties.mailing_city[i]
    city = remove_punctuation(city)
    city = city.replace('township', '')
    city = city.replace('BCH', 'BEACH')
    if city[:3] == 'ST ':
        city = 'SAINT ' + city[3:]
    elif city[:3] == 'FT ':
        city = 'FORT ' + city[3:]
    elif city[:3] == 'MT ':
        city = 'MOUNT ' + city[3:]
    elif city[:3] == 'PT ':
        city = 'POINT ' + city[3:]
    elif city[:2] == 'N ':
        city = 'NORTH ' + city[2:]
    elif city[:2] == 'S ':
        city = 'SOUTH ' + city[2:]
    elif city[:2] == 'E ':
        city = 'EAST ' + city[2:]
    elif city[:2] == 'W ':
        city = 'WEST ' + city[2:]
    elif city[:3] == 'NW ':
        city = 'NORTHWEST ' + city[3:]
    elif city[:3] == 'NE ':
        city = 'NORTHEAST ' + city[3:]
    elif city[:3] == 'SW ':
        city = 'SOUTHWEST ' + city[3:]
    elif city[:3] == 'SE ':
        city = 'SOUTHEAST ' + city[3:]
    word_list = city.split()
    w = str()
    for thing in word_list:
        if w.isnumeric() is False:
            w = w + ' ' + w.strip()
        else:
            continue
    city = w.strip()
    city = word_replacement(city, properties.site_city[i])
    properties.mailing_city[i] = city

C:\Users\robin\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
<ipython-input-4-e2e7e9a19930>:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  properties.mailing_city[i] = city


In [5]:
sales = properties[properties['trans_id'].notnull()]
sales.drop(columns=['property_use', 'owner_2', 'owner_3', 'market_value', 'assessed_value', 'value_method', 
                    'perc_complete', 'seller_2', 'status', 'reason', 'improved'], inplace=True)
sales.shape

C:\Users\robin\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


(42971, 95)

In [7]:
sales.to_csv('DuvalHouseSales.csv')

In [9]:
for col in properties.columns:
    print(col)

RE
owner_1
owner_2
owner_3
mailing_address
mailing_city
mailing_state
property_use
subdivision
neighborhood
value_method
cap_base_yr
market_value
assessed_value
building_value
just_value
school_taxable
county_taxable
sjrwmd_taxable
tax_district
lot_sf
char_descr
canal
golf
intracoastal
lake
misc_waterway
ocean_front
ortega_river
ribault_river
stjohns_river
trout_river
building
type_descr
style
class
quality
actual_yr_built
effec_yr_built
perc_complete
value
heated_sf
stories
bedrooms
baths
rooms
max_units_per_acre
use_canal
use_golf
use_marsh
use_ocean
use_pond
use_river
land_val
use_types
trans_id
status
reason
sale_date
seller_1
seller_2
improved
price
boat_cv
boatcv_ppu
boatcv_yr
carport
carport_ppu
carport_yr
cov_patio
covpatio_ppu
covpatio_yr
deck
deck_ppu
deck_yr
dock
dock_ppu
dock_yr
fireplace
fireplace_ppu
fireplace_yr
pool
pool_ppu
pool_yr
scr_porch
scrporch_ppu
scrporch_yr
screen_encl
screenencl_ppu
screenencl_yr
shed
shed_ppu
shed_yr
sun_rm
sunrm_ppu
sunrm_yr
additions
addit

C:\Users\robin\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


(42922, 92)

In [7]:
sales.shape

(42922, 102)

In [21]:
for column in df.columns:
    print(column, '|', df[column].dtype, '|', df[column].count())

RE | object | 252463
owner_1 | object | 252463
owner_2 | object | 126671
owner_3 | object | 10736
mailing_address | object | 252463
mailing_city | object | 252463
mailing_state | object | 252463
property_use | int64 | 252463
subdivision | object | 252463
neighborhood | float64 | 252463
val_method | int64 | 252463
cap_base_yr | int64 | 252463
market_val | int64 | 252463
assessed_val | int64 | 252463
building_val | int64 | 252463
just_val | int64 | 252463
school_taxable | int64 | 252463
county_taxable | int64 | 252463
sjrwnd_taxable | int64 | 252463
taxing_district | object | 252463
square_feet | int64 | 252463
char_1 | object | 20616
char_2 | object | 734
char_3 | object | 3
building | int64 | 252463
type_descr | object | 252463
style | int64 | 252463
class | int64 | 252463
quality | float64 | 252463
actual_yr_built | int64 | 252463
effec_yr_built | int64 | 252463
perc_complete | float64 | 252463
value | int64 | 252463
heated_sf | int64 | 252463
stories | float64 | 252463
bedrooms | flo

In [26]:
for i in range(len(properties.mailing_address)):
    address = properties.mailing_address[i]
    address = remove_punctuation(address)
    address_words = address.split()
    w = str()
    for word in address_words:
        if fuzz.WRatio(word.strip(),'AVENUE') >= 90:
            w = w + ' ' + 'AVENUE'
        elif fuzz.WRatio(word.strip(),'BEACH') >= 90:
            w = w + ' ' + 'BEACH'
        elif fuzz.WRatio(word.strip(),'BOULEVARD') >= 90:
            w = w + ' ' + 'BOULEVARD'
        elif fuzz.WRatio(word.strip(),'CIRCLE') >= 90:
            w = w + ' ' + 'CIRCLE'
        elif fuzz.WRatio(word.strip(),'COURT') >= 90:
            w = w + ' ' + 'COURT'
        elif fuzz.WRatio(word.strip(),'DRIVE') >= 90:
            w = w + ' ' + 'DRIVE'
        elif fuzz.WRatio(word.strip(),'EXPRESSWAY') >= 90:
            w = w + ' ' + 'EXPRESSWAY'
        elif fuzz.WRatio(word.strip(),'GATEWAY') >= 90:
            w = w + ' ' + 'GATEWAY'
        elif fuzz.WRatio(word.strip(),'HARBOR') >= 90:
            w = w + ' ' + 'HARBOR'
        elif fuzz.WRatio(word.strip(),'HIGHWAY') >= 90:
            w = w + ' ' + 'HIGHWAY'
        elif fuzz.WRatio(word.strip(),'ISLAND') >= 90:
            w = w + ' ' + 'ISLAND'
        elif fuzz.WRatio(word.strip(),'ISLE') >= 90:
            w = w + ' ' + 'ISLE'
        elif fuzz.WRatio(word.strip(),'JUNCTION') >= 90:
            w = w + ' ' + 'JUNCTION'
        elif fuzz.WRatio(word.strip(),'LANE') >= 90:
            w = w + ' ' + 'LANE'
        elif fuzz.WRatio(word.strip(),'PARKWAY') >= 90:
            w = w + ' ' + 'PARKWAY'
        elif fuzz.WRatio(word.strip(),'PLAZA') >= 90:
            w = w + ' ' + 'PLAZA'
        elif fuzz.WRatio(word.strip(),'POINT') >= 90:
            w = w + ' ' + 'POINT'
        elif fuzz.WRatio(word.strip(),'ROAD') >= 90:
            w = w + ' ' + 'ROAD'
        elif fuzz.WRatio(word.strip(),'ROUTE') >= 90:
            w = w + ' ' + 'ROUTE'
        elif fuzz.WRatio(word.strip(),'SQUARE') >= 90:
            w = w + ' ' + 'SQUARE'
        elif fuzz.WRatio(word.strip(),'SUITE') >= 90:
            w = w + ' ' + 'SUITE'
        elif fuzz.WRatio(word.strip(),'VILLAGE') >= 90:
            w = w + ' ' + 'VILLAGE'
        else:
            w = w + ' ' + word.strip()
    address = w.strip()
    address = address.replace('NORTHWEST', 'NW')
    address = address.replace('NORTHEAST', 'NE')
    address = address.replace('SOUTHWEST', 'SW')
    address = address.replace('SOUTHEAST', 'SE')
    address = address.replace('NORTH', 'N')
    address = address.replace('SOUTH', 'S')
    address = address.replace('WEST', 'W')
    address = address.replace('EAST', 'E')
    address = address.replace('BCH', 'BEACH')
    address = address.replace('AVENUE', 'AVE')
    address = address.replace(' AV ', ' AVE ')
    address = address.replace('BOULEVARD', 'BLVD')
    address = address.replace('CIRCLE', 'CIR')
    address = address.replace('COURT', 'CT')
    address = address.replace('DRIVE', 'DR')
    address = address.replace('HIGHWAY', 'HWY')
    address = address.replace('JUNCTION', 'JCT')
    address = address.replace(' LANE', ' LN')
    address = address.replace('PARKWAY', 'PKWY')
    address = address.replace('PLAZA', 'PLZ')
    address = address.replace('ROAD', 'RD')
    address = address.replace('ROUTE', 'RTE')
    address = address.replace('SQUARE', 'SQ')
    address = address.replace('STREET', 'ST')
    address = address.replace('SUITE', 'STE')
    properties.mailing_address[i] = address

<ipython-input-26-c3f5b73a4797>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  properties.mailing_address[i] = address


KeyboardInterrupt: 